# 언어데이터과학 25강 (2023-12-03) 옛한글 처리

## 할 일

### 오늘의 목표

현대 한국어 표기에 쓰이지 않는 옛한글을 포함하여 한국어 음절 유형을 분류할 수 있다.
  > ᄣᆞᆯ -> CCCVC, ᄎᆔ -> CGVV


### 심화 과제

임의의 한글 자모의 연쇄를 한글 음절로 합성할 수 있다.
  > ㅂㅏㅂㅅㅜㄷㄱㅏㄹㅏㄱ -> 밥숟가락

## 옛한글 처리 방식



### 과거

유니코드가 널리 보급되기 전 2000년대까지는 옛한글을 처리할 때 대체로 PUA 방법을 사용했다.

+ PUA(Private Use Areas): 유니코드에서 코드가 할당되지 않은 문자 영역.

PUA에 옛한글 문자를 임의로 할당했으므로, 유니코드에서 옛한글을 처리할 수 없다.

In [1]:
spy = '\ue13d'
print(spy)
# from unicodedata import name
# print(name(spy)) # ValueError




예를 들어 위의 문자는 PUA에 할당된 옛한글이다. 이런 문자는 옛한글을 특별히 지원하는 폰트로만 볼 수 있다는 한계가 있다.

가장 쉽게 다운로드할 수 있는 폰트로는 네이버 한글한글 아름답게 프로젝트에서 공개한 "나눔명조옛한글"과 "나눔바른고딕옛한글"이 있다. 아래의 링크에서 미리보기 및 다운로드가 가능하다.

https://hangeul.naver.com/font/nanum

![](./img/nanum.png)

### 현재

PUA 방식은 지금도 일부에서 쓰고 있으나, 유니코드가 보급된 2010년대 이후로는 "첫가끝" 방식을 권장한다.

+ "첫가끝"(첫소리-가운뎃소리-끝소리): 유니코드에 할당된 초성 문자, 중성 문자, 종성 문자를 조합하여 하나의 음절처럼 모아서 보여주는 방법.

In [2]:
leading = 'ᄀ'
vowel = 'ᅮ'
trailing = 'ᇚ'
print(leading+vowel+trailing)

구ᇚ


유니코드에서 사용 가능한 한글 자모 초성 문자, 중성 문자, 종성 문자의 목록은 아래의 표에서 볼 수 있다.

![](https://upload.wikimedia.org/wikipedia/commons/b/bd/Hangul_jamo_characters_in_Unicode.svg)

## `unicodedata` 모듈을 사용한 Python 옛한글 처리

이번 실습에서는 첫가끝 방식으로 옛한글 음절과 현대 한글 음절을 일관성 있게 처리할 것이다.

우선 필요한 모듈과 함수를 가져오자.

In [3]:
from unicodedata import lookup, normalize, name
import re

### 유니코드 문자의 두 가지 정보: 코드 포인트와 이름


옛한글 처리에서 먼저 맞닥뜨리는 문제는 옛한글에 사용되는 자모를 일반적인 한글 키보드로 쉽게 입력하지 못한다는 것이다.

Python에서는 유니코드의 문자 정보를 활용하여 이 문제를 해결할 수 있다.

유니코드 문자 정보로는 코드 포인트와 이름이 있다.

코드 포인트를 문자로 변환해 주는 함수는 `chr()`, 이름을 문자로 변환해 주는 함수는 `unicodedata.lookup()`이다.

![](./img/kiyeok.png)

예를 들어 한글 초성 'ㄱ'의 코드 포인트는 16진수 1100, 이름은 'HANGUL CHOSEONG KIYEOK'이므로, 아래와 같이 입력할 수 있다.

In [4]:
kiyeok1 = chr(0x1100) # 0x1100: 16진수 1100
kiyeok2 = lookup('HANGUL CHOSEONG KIYEOK')
print(kiyeok1, kiyeok2)
print(kiyeok1 == kiyeok2)

ᄀ ᄀ
True


위와 같이 코드 포인트를 `chr()` 함수의 인자로 넣어 주거나 이름을 `unicodedata.lookup()` 함수의 인자로 넣어 주면 해당하는 문자를 얻는다.

코드 포인트와 이름을 모른다면 아래의 문서에서 찾아볼 수 있다.


#### 한글 자모(초성, 중성, 종성)의 코드 포인트와 이름 알아보기

+ https://www.unicode.org/charts/PDF/U1100.pdf HANGUL JAMO
+ https://www.unicode.org/charts/PDF/UA960.pdf HANGUL JAMO EXTENDED-A
+ https://www.unicode.org/charts/PDF/UD7B0.pdf HANGUL JAMO EXTENDED-B

### `lookup()` 함수로 옛한글 입력하기

각 문자의 코드 포인트는 외우거나 매번 찾아보아야 해서 번거로우므로, 이 실습에서는 사람이 좀 더 추측하기 쉬운 이름을 사용할 것이다.

+ `unicodedata.lookup()`: 유니코드에서 정의된 문자의 이름을 인자로 받아서 해당하는 문자를 돌려주는 함수

이 함수를 활용하여 옛한글 자모가 포함된 음절을 입력해 보자.

In [5]:
# CHOSEONG, JUNGSEONG, JONGSEONG 순서가 맞아야 하나의 음절로 모아서 보여줌
onset1 = lookup('HANGUL CHOSEONG PIEUP-SIOS-TIKEUT')
nucleus1 = lookup('HANGUL JUNGSEONG ARAEA')
coda1 = lookup('HANGUL JONGSEONG RIEUL')
print(onset1+nucleus1+coda1)

ᄣᆞᆯ


In [6]:
onset2 = lookup('HANGUL CHOSEONG CHIEUCH')
nucleus2 = lookup('HANGUL JUNGSEONG YU-I')
print(onset2+nucleus2)

ᄎᆔ


In [7]:
onset3 = lookup('HANGUL CHOSEONG KIYEOK')
nucleus3 = lookup('HANGUL JUNGSEONG U')
coda3 = lookup('HANGUL JONGSEONG MIEUM-KIYEOK')
print(onset3+nucleus3+coda3)

구ᇚ


### `normalize()` 함수로 한글 음절 분해하기

위에서 옛한글을 자모 문자의 조합으로 입력할 수 있음을 알아보았다.

그런데 현대 한글의 경우 하나의 완성된 음절이 하나의 문자로 처리된다.

In [8]:
syllable = '꺇'
print(syllable, len(syllable))

꺇 1


예를 들어 위에서 '꺇'이라는 음절은 `len()` 함수로 길이를 확인했을 때 1의 값이 나온다. 즉, '꺇' 자체가 한 개의 문자라는 것이다.

그런데 '꺇'의 음절 유형이 CGVCC라는 사실을 알아내기 위해서는 이 음절이 세 개의 자모 'ᄁ', 'ᅣ', 'ᆪ'로 이루어져 있다는 정보가 필요하다.

또한 옛한글과 현대 한글을 일관된 방식으로 처리하려면 현대 한글 역시 옛한글처럼 자모 문자의 조합으로 표시해 주어야 한다.

다행히 유니코드에서 '꺇'이라는 하나의 문자가 'ᄁ', 'ᅣ', 'ᆪ' 세 개의 문자로 분해될 수 있다는 정보가 들어 있다.

이 정보를 통해 한글 음절 문자나 자모 문자들의 연쇄를 정규화(normalize)할 수 있다. 정규화 방식은 아래의 네 가지가 있다.

+ NFD: Normalization Form (Canonical) D(ecomposition)
+ NFC: Normalization Form (Canonical) C(omposition)
+ NFKD: Normalization Form Compatibility D(ecomposition)
+ NFKC: Normalization Form Compatibility C(omposition)

한글 음절 한 문자를 자모 문자들로 분해(decompose)하려면 'NFD' 또는 'NFKD' 방식으로 정규화하면 된다.

매번 정규화 방식을 명시하기 귀찮으므로, 문자를 분해하는 함수를 만들어 놓자.

In [9]:
def decompose(char):
  return normalize('NFKD', char)

아래의 예시에서 '꺇'을 분해한 결과는, 눈으로 보기에는 분해 이전의 '꺇'과 모양이 똑같다.

그러나 `len()` 함수를 통해 길이를 확인하면 1이 아니라 3이 나온다. 즉, 1개의 문자처럼 합쳐져 있지만 사실은 3개의 문자라는 것이다.

In [10]:
jamos = decompose('꺇')
print(jamos, len(jamos), tuple(jamos))

꺇 3 ('ᄁ', 'ᅣ', 'ᆪ')


### `name()` 함수로 문자 이름 찾기

다음으로 `unicodedata.lookup()` 함수의 역함수에 관해 알아볼 것이다.

`unicodedata.name()` 함수는 1개의 문자를 인자로 받아 그 문자의 유니코드 이름을 돌려준다.

한글 (2벌식) 키보드에서 'ㄱ' 키를 눌렀을 때 입력되는 문자의 이름을 확인해 보자.

In [11]:
char = 'ㄱ'
print(name(char))

HANGUL LETTER KIYEOK


위의 결과에서 주목해야 할 점은 이름의 두 번째 단어가 'CHOSEONG'이나 'JONGSEONG'이 아닌 'LETTER'라는 것이다.

즉, 한글 자모 'ㄱ'에 해당하는 유니코드 문자는 모두 세 개이고, 각각의 코드 포인트와 이름은 아래와 같다.

+ U+1100, HANGUL CHOSEONG KIYEOK (초성 문자)
+ U+11A8, HANGUL JONGSEONG KIYEOK (종성 문자)
+ U+3131, HANGUL LETTER KIYOEK (호환성 문자)

한글 음절 1문자를 분해하면 초성 문자와 종성 문자가 나오고, 한글 키보드에서 1개의 키를 누르면 호환성 문자가 나온다.

앞서 만든 `decompose()` 함수의 반환값으로 나오는 문자들의 이름을 확인해 보자.

In [12]:
def print_jamo_name(syl:str):
  print(syl, name(syl))
  print('----------')
  for jamo in decompose(syl):
    print(jamo, hex(ord(jamo)), name(jamo))

In [13]:
print_jamo_name('아') # V

아 HANGUL SYLLABLE A
----------
ᄋ 0x110b HANGUL CHOSEONG IEUNG
ᅡ 0x1161 HANGUL JUNGSEONG A


In [14]:
print_jamo_name('뷁') # CGVCC

뷁 HANGUL SYLLABLE BWELG
----------
ᄇ 0x1107 HANGUL CHOSEONG PIEUP
ᅰ 0x1170 HANGUL JUNGSEONG WE
ᆰ 0x11b0 HANGUL JONGSEONG RIEUL-KIYEOK


이 실습의 목표인 음절 유형 분류를 달성하기 위해서는 초성 이름이 'IEUNG'으로 끝나는지, 중성 이름의 마지막 단어에 활음(glide)이 있는지 등의 정보가 필요하다.

앞으로 이 정보를 자주 사용할 것 같으므로, 음절이나 자모의 로마자에 해당하는 마지막 단어만 추출하는 함수를 미리 만들어 놓자.

In [15]:
def romanize(hangul):
  hname = name(hangul)
  assert hname.startswith('HANGUL')
  return hname.split()[-1]

몇 가지 한글 문자의 로마자 추출 결과를 확인해 보자. 특히 겹자음의 경우 하이픈('-')으로 이름이 묶여 있다는 데 유의하자.

In [16]:
string = '뷁ㄱㄳㅃㅇㅏㅑㅟㅞ'
for char in string:
    print(char, romanize(char))

뷁 BWELG
ㄱ KIYEOK
ㄳ KIYEOK-SIOS
ㅃ SSANGPIEUP
ㅇ IEUNG
ㅏ A
ㅑ YA
ㅟ WI
ㅞ WE


## 한글 음절 유형 판별하기 (예비)

모든 현대 한글 음절과 자모 풀어쓰기, 일부 옛한글에 대해 음절 유형을 판별할 수 있다.

```
>>> print(toy('굼'))
CVC
>>> print(toy('ㄱㅜㅁ'))
CVC
>>> print(toy('구ᇚ'))
CVCC
```


### 예비 절차



지금까지 `unicodedata` 모듈의 기능을 살펴보고 몇 가지 함수를 만들었다.

이제 한글 음절의 유형을 판별해 보자.

가장 간단하고 명백한 경우 아래와 같은 함수를 작성할 수 있다.

In [17]:
def toy(syllable):
    # 결과물을 초기화한다
    output = ''

    # 음절을 초성, 중성, 종성으로 분해한다
    dec = decompose(syllable)
    onset = dec[0] # 초성
    nucleus = dec[1] # 중성
    coda = dec[2:] # 종성
    
    # 초성: 'ㅇ'이면 초성이 없고, 'ㅇ'이 아니면 C(자음 1개)
    # 'ㅇ'의 로마자는 'IEUNG'이다
    if romanize(onset) == 'IEUNG':
        pass
    else:
        output += 'C'

    # 중성: 활음으로 시작하면 GV(이중모음), 활음으로 시작하지 않으면 V(단모음)
    # 활음의 로마자는 'W'나 'Y'로 시작한다
    if romanize(nucleus).startswith(('W', 'Y')):
        output += 'GV'
    else:
        output += 'V'
    
    # 종성: 비어 있으면 종성이 없고, 단자음이면 C, 겹자음이면 CC
    # 겹자음은 로마자에 '-'가 들어 있다
    if coda == '':
        pass
    elif romanize(coda).isalpha():
        output += 'C'
    else:
        output += 'CC'
    
    # 최종 결과를 반환한다
    return output

사실 위와 같이 작성하기만 해도 기본적인 상황에 적용하는 데는 문제가 없다.

In [18]:
syllables = ('아', '가', '야', '뷁')
for syllable in syllables:
    print(syllable, toy(syllable))

아 V
가 CV
야 GV
뷁 CGVCC


위와 같이 현대 한글 음절의 경우는 잘 작동한다.

In [19]:
syllables_jamo = ('ㅂㅏㄺ', 'ㄴㅔ', 'ㅇㅛ')
for syllable in syllables_jamo:
    print(syllable, toy(syllable))

ㅂㅏㄺ CVCC
ㄴㅔ CV
ㅇㅛ GV


심지어 자모로 풀어써서 넣어도 잘 작동한다.

In [20]:
syllables_archaic_good = ('구ᇚ', 'ᄃᆞᆯ')
for syllable in syllables_archaic_good:
    print(syllable, toy(syllable))

구ᇚ CVCC
ᄃᆞᆯ CVC


일부 옛한글에 대해서도 잘 작동한다. 여기까지 보면 괜찮은 것 같다.

하지만 잘 작동하지 않는 경우도 있다.

### 문제점

In [21]:
syllables_archaic_bad = ('ᄣᆞᆯ', 'ᄎᆔ')
for syllable in syllables_archaic_bad:
    print(syllable, toy(syllable))

ᄣᆞᆯ CVC
ᄎᆔ CGV


우선 위와 같이 'ᄣᆞᆯ'(CCCVC), 'ᄎᆔ'(CGVV)와 같은 옛한글은 음절 유형을 잘못 추출하는 경우도 있다.

In [22]:
toy('ㅂㅂㅂ')

'CVC'

이뿐만 아니라 'ㅂㅂㅂ'처럼 음절을 이루지 않는 한글 문자열도 음절 유형이 추출되어 버린다.

이후의 내용에서는 아래의 두 문제를 해결하고자 한다.

1. 임의의 옛한글 음절이 주어졌을 때 음절 유형을 정확하게 판별한다.
2. 인자의 값이 정확히 한글 음절을 이루는 경우에만 음절 유형을 반환한다.

## 한글 음절 유형 판별하기 (기본)

1음절로 표현되는 모든 옛한글과 자모 풀어쓰기에 대해 음절 유형을 판별할 수 있다.

```
>>> print(get_syllable_type('ᄣᆞᆯ'))
CCCVC
```

### 자모 단위로 유형 판별하기

#### 준비물

음절 유형을 더 정교하게 판별하기 위해 음절을 초성, 중성, 종성으로 분리해서 각각의 유형을 분류하는 방식으로 접근하고자 한다.

이를 위해 우선 한글 자모가 주어졌을 때 초성인지 아닌지 등을 판별하는 함수를 만들어 놓자.

In [23]:
def is_choseong(jamo):
    return name(jamo).startswith('HANGUL CHOSEONG')

def is_jungseong(jamo):
    return name(jamo).startswith('HANGUL JUNGSEONG')

def is_jongseong(jamo):
    return name(jamo).startswith('HANGUL JONGSEONG')

In [24]:
# 예시: 세 종류의 'ㄱ'

choseong, _, jongseong = decompose('각')
letter = 'ㄱ'

kiyeoks = choseong, jongseong, letter
print(kiyeoks)
print(tuple(map(romanize, kiyeoks)))
print(tuple(map(is_choseong, kiyeoks)))
print(tuple(map(is_jongseong, kiyeoks)))

('ᄀ', 'ᆨ', 'ㄱ')
('KIYEOK', 'KIYEOK', 'KIYEOK')
(True, False, False)
(False, True, False)


다음으로 임의의 한글 자음(CHOSEONG, JONGSEONG, LETTER) 문자를 초성이나 종성으로, 모음(JUNGSEONG, LETTER) 문자를 중성으로 변환하는 함수를 만들자.

In [25]:
def to_choseong(jamo):
    try:
        jname = name(jamo)
        return lookup(re.sub('LETTER|JONGSEONG', 'CHOSEONG', jname))
    except:
        return jamo

def to_jungseong(jamo):
    try:
        jname = name(jamo)
        return lookup(jname.replace('LETTER', 'JUNGSEONG'))
    except:
        return jamo

def to_jongseong(jamo):
    try:
        jname = name(jamo)
        return lookup(re.sub('LETTER|CHOSEONG', 'JONGSEONG', jname))
    except:
        return jamo

#### 초성 유형 식별하기

이제 1음절을 이루는 초성, 중성, 종성의 유형을 각각 판별할 것이다.

여기에서는 인자의 값이 정확히 한글 음절을 이루는 경우에만 음절 유형을 반환하고자 한다.

그러므로 초성을 위한 함수에서는 초성(CHOSEONG)이나 호환성(LETTER) 문자가 들어오는 경우에만 자음 유형을 판별하고, 이외의 문자는 오류를 일으키자.

앞서 `toy()` 함수에서는 초성 위치의 문자가 'ㅇ'이 아닌 경우 'C'를 반환하였으나, 이번에는 겹자음을 고려하기 위해 '-'의 개수를 센다.

하이픈이 0개인 경우 단자음(C), 1개인 경우 겹자음(CC), 2개의 경우 삼겹자음(CCC)이 되도록 반환값을 설정하면 된다.

In [26]:
def get_onset_type(jamo):
  # 한글 초성 문자로 변환할 수 없는 경우 오류를 일으키고 중단한다
  if not is_choseong(to_choseong(jamo)) or is_jongseong(jamo):
    raise ValueError('Not a valid Hangul choseong(onset, leading consonant) character')

  roman = romanize(jamo)

  # 문자가 'ㅇ'인 경우 C가 없다
  if romanize(jamo) == 'IEUNG':
    return ''
  
  # 옛한글 대응: '-'로 결합한 자음의 개수만큼 C를 내보낸다
  return (roman.count('-')+1) * 'C'

이 함수를 자음 문자에 적용해 보면 아래의 여러 예시와 같이 의도대로 잘 작동하는 것을 볼 수 있다.

In [27]:
print(get_onset_type(lookup('HANGUL CHOSEONG KIYEOK'))) # 초성 문자
print(get_onset_type(lookup('HANGUL LETTER KIYEOK'))) # 호환성 문자

C
C


In [28]:
# print(get_onset_type(lookup('HANGUL JONGSEONG KIYEOK'))) # 종성 문자는 ValueError를 일으킨다

In [29]:
print(get_onset_type(lookup('HANGUL CHOSEONG PIEUP-SIOS-TIKEUT'))) # 옛한글 3중자음

CCC


### 중성 유형 식별하기

중성의 경우도 `toy()`의 내용을 개선하여 인자의 유형을 제한하고, 겹모음을 허용할 수 있도록 한다.

In [30]:
def get_nucleus_type(jamo, GLIDES=re.compile(r'^[WY]')):
  # 한글 중성 문자로 변환할 수 없는 경우 오류를 일으키고 중단한다
  if not is_jungseong(to_jungseong(jamo)):
    raise ValueError('Not a valid Hangul jungseong(nucleus, vowel) character')

  roman = romanize(jamo)
  output = ''
  
  # 옛한글 대응: '-'로 분리된 모음의 개수만큼 V 혹은 GV를 내보낸다
  for vowel in roman.split('-'):
    output += ('GV' if GLIDES.search(vowel) else 'V')
  
  return output

In [31]:
print(get_nucleus_type(lookup('HANGUL JUNGSEONG YU-I')))
print(get_nucleus_type(lookup('HANGUL LETTER YU-I')))

GVV
GVV


In [32]:
# print(get_nucleus_type('ㅊ'))

### 종성 유형 식별하기

종성 유형은 'ㅇ'이 자음 음가를 가지고 있으므로 초성 유형보다 조금 더 간단하게 판별할 수 있다.

In [33]:
def get_coda_type(jamo):
  # 한글 종성 문자로 변환할 수 없는 경우 오류를 일으키고 중단한다
  if not is_jongseong(to_jongseong(jamo)) or is_choseong(jamo):
    raise ValueError('Not a valid Hangul jongseong(coda, trailing consonant) character')

  roman = romanize(jamo)

  # 옛한글 대응: '-'로 결합한 자음의 개수만큼 C를 내보낸다
  return (roman.count('-')+1) * 'C'

In [34]:
print(get_coda_type(lookup('HANGUL JONGSEONG YESIEUNG')))
print(get_coda_type(lookup('HANGUL LETTER YESIEUNG')))

C
C


In [35]:
# print(get_coda_type(lookup('HANGUL CHOSEONG YESIEUNG')))

### 음절 여부 판별하기

실제 음절에 대해서만 음절 유형을 판별하기 위해서는, 주어진 문자열이 한글 음절인지 아닌지를 판별해야 한다.

우선 1개 문자가 한글 음절인지 아닌지는 문자의 유니코드 이름으로 쉽게 판별할 수 있다.

In [36]:
def is_syllable(char):
  try:
    return name(char).startswith('HANGUL SYLLABLE')
  except ValueError:
    return False
  except TypeError:
    return False

In [37]:
print(is_syllable('앍')) # 음절 문자임
print(is_syllable('ㅇ')) # 음절 문자가 아님
print(is_syllable('ㅏ')) # 음절 문자가 아님

True
False
False


그런데 1개 음절이 자모로 분해된 문자열의 경우는 대응할 수 없다.

In [38]:
print(is_syllable(decompose('앍')))

False


자모의 연쇄가 한글 음절을 이루는지 아닌지를 판별하는 함수를 추가로 만들자.

첫 번째 문자를 초성으로 변환할 수 있고, 두 번쨰 문자를 중성으로 변환할 수 있고, 세 번째 문자가 문자가 존재하는 경우 종성으로 변환할 수 있는 경우 True, 이외의 경우는 모두 False를 반환하면 된다.

In [39]:
def is_jamo_syllable(string):
    try:
        onset, nucleus = string[0:2]
        result = is_choseong(to_choseong(onset)) and is_jungseong(to_jungseong(nucleus))
        coda = string[2:]
        if coda:
            result = result and is_jongseong(to_jongseong(coda))
    except:
        return False
    
    return result

In [40]:
print(is_jamo_syllable('ㄱㅏㅁ'))
print(is_jamo_syllable('ㄱㅏㅁ4'))
print(is_jamo_syllable('ㄱㅏㅁㅁ'))
print(is_jamo_syllable('ᄣᆞᆯ'))

True
False
False
True


두 함수를 조합하여 주어진 문자열이 한글 음절인지 아닌지를 판별하는 함수를 만들 수 있다.

In [41]:
def is_any_syllable(string):
    return is_jamo_syllable(string) or is_syllable(string)

In [42]:
print(is_any_syllable('ㅇ'))
print(is_any_syllable('앆'))
print(is_any_syllable(decompose('앆')))
print(is_any_syllable('ㅇㅏ'))
print(is_any_syllable('ㅇㅏㄲ'))
print(is_any_syllable('ㅇㅏㅇㅇ'))

False
True
True
True
True
False


위의 예시에서 볼 수 있듯이 '앆', decompose('앆'), 'ㅇㅏㄲ'과 같은 다양한 음절 표현 방식을 모두 판별할 수 있다.

### 최종: 음절 유형 식별하기

이제 한글 1음절을 표현하는 문자열이 주어졌을 때 해당 음절의 유형을 식별할 수 있다.

아래의 코드에서는 'ㅋㅋㅋ' 등 더 다양한 경우에 사용할 수 있도록 자음 1개나 모음 1개의 경우에도 'C' 또는 'V'를 반환하는 내용을 추가했다.

In [43]:
def get_syllable_type(syl):
  '''
  >>> get_syllable_type('ㅂ')
  'C'
  >>> get_syllable_type('ㅏ')
  'V'
  >>> get_syllable_type('밤')
  'CVC'
  >>> get_syllable_type('ㅂㅏ')
  'CV'
  '''
  # HANGUL SYLLABLE: need to be decomposed
  if is_any_syllable(syl):
    dec = decompose(syl)

  # Consonant jamo only
  elif is_choseong(to_choseong(syl)):
    return get_onset_type(syl)

  elif is_jongseong(to_jongseong(syl)):
    return get_coda_type(syl)

  # Vowel jamo only
  elif is_jungseong(to_jungseong(syl)):
    return get_nucleus_type(syl)

  else:
    return None
  
  onset, nucleus = dec[0:2]
  output = get_onset_type(onset) + get_nucleus_type(nucleus)

  coda = dec[2:]
  if coda:
    output += get_coda_type(coda)
  
  return output

In [44]:
consonant = 'ㄸ'
print(get_syllable_type(consonant))

C


In [45]:
vowel = 'ㅑ'
print(get_syllable_type(vowel))

GV


In [46]:
print(get_syllable_type('3')) # None

None


In [47]:
syl = 'ㄲㅑㄳ'
print(get_syllable_type(syl))

CGVCC


In [48]:
syl1 = 'ᄣᆞᆯ'
print(get_syllable_type(syl1))

CCCVC


In [49]:
syl2 = 'ᄎᆔ'
print(get_syllable_type(syl2))

CGVV


In [50]:
print(get_syllable_type('ᄇᆡᆨ'))

CVVC


겹자음 초성이나 겹모음 중성을 가진 경우에도 잘 작동하는 것을 확인할 수 있다.

여기에서 더 확장된 내용을 알고 싶은 경우 아래를 읽고 문제를 풀어 보자. (관심이 없으면 읽지 않아도 된다.)

## [숙제14심화] 한글 음절 유형 판별하기 (확장) [주의: 어려움]

임의의 한글 문자열이 주어졌을 때 (1) 문자열 내에서 가능한 음절을 모두 조합하고 (2) 모든 음절 유형을 추출할 수 있다.

```
>>> print(syllabify_many('ㅺㅜㄹ1ㅁㅡㄹ'))
('ᄭᅮᆯ', '1', '믈')
>>> print(get_syllable_types('ㅺㅜㄹ1ㅁㅡㄹ'))
('CCVC', 'CVC')
```

(1)과 (2) 두 가지 목표를 달성하기 위해 아래의 세 가지 함수를 활용할 수 있다.

### 재료 1. 주어진 문자열이 한글로만 이루어져 있는지 아닌지 판별하는 함수

In [51]:
def ishangul(string:str):
    if string:
        return all(name(char).startswith('HANGUL') for char in string)
    else:
        return False

In [52]:
strings = ('ㄱ', 'ㅏ', 'ㅏㄴ', '만', '만ㄴ', '만1')
for string in strings:
    print(string, ishangul(string))

ㄱ True
ㅏ True
ㅏㄴ True
만 True
만ㄴ True
만1 False


### 재료 2. 1음절의 자모 풀어쓰기를 1개의 음절 문자로 조합하는 함수

In [53]:
def syllabify_one(jamos):
    output = ''
    if 2 <= len(jamos) <= 3:
        output += to_choseong(jamos[0])
        output += to_jungseong(jamos[1])
    if len(jamos) == 3:
        output += to_jongseong(jamos[2])
    
    if output:
        return normalize('NFKC', output)
    else:
        return jamos

In [54]:
print(syllabify_one('ㅇㅏㄺ')) # 조합 가능
print(syllabify_one('ㅇㅏㄻㅁ')) # 조합 불가능
print(syllabify_one('1234')) # 조합 불가능

앍
ㅇㅏㄻㅁ
1234


### 재료 3. 한글 자모 문자(CHOSEONG, JUNGSEONG, JONGSEONG)를 한글 호환성 문자(LETTER)로 변환하는 함수

In [55]:
def to_letter(jamo):
    try:
        jname = name(jamo)
        return lookup(re.sub('(?:CHO|JUNG|JONG)SEONG', 'LETTER', jname))
    except:
        return jamo

In [56]:
def to_letters(jamos):
    return ''.join(to_letter(jamo) for jamo in jamos)

### Q1. 임의의 길이의 한글 문자열에서 조합할 수 있는 모든 음절을 조합하는 함수 (최대 2점)

문자열 'ㅂㅏㅂㅏㅁㅂㅏ'를 튜플 ('바', '밤', '바')로 변환할 수 있도록 아래의 함수를 완성하라.

In [57]:
def syllabify_many(jamos):
    jamos = to_letters(jamos)

    # [hw14adv] Q1
    # DO SOMETHING HERE
    # DO SOMETHING HERE
    
    return tuple() # EDIT THIS LINE

#### Q1 힌트

1. 'ㅂㅏ', 'ㅂㅏㅁ' 등 1개 음절을 풀어쓴 문자열을 매치시킬 수 있는 정규표현식을 작성한다.
2. 모든 한글 자모를 호환성(LETTER) 문자로 통일하면 정규표현식을 좀 더 쉽게 작성할 수 있다.

#### Q1 채점 기준

1. 'ㅂㅏㅂㅏㅁㅂㅏ'처럼 현대 한글 자모만으로 이루어진 문자열을 음절화할 수 있다. (1점)
2. 옛한글 자모를 포함한 문자열을 음절화할 수 있다. (+0.5점)
3. ':ㄱㅣㅁ4ㅂㅏㅂ'의 ':'처럼 한글이 아닌 문자도 반환값에 포함한다. (+0.5점)

In [58]:
# 요구사항 1 (1점)
jamos = 'ㅂㅏㅂㅏㅁㅂㅏ'
print(syllabify_many(jamos))

('바', '밤', '바')


In [59]:
# 요구사항 2 (+0.5점)
jamos = 'ㅊㆌㄱㅐㄱ'
print(syllabify_many(jamos))

('ᄎᆔ', '객')


In [60]:
# 요구사항 3 (+0.5점)
jamos = ':ㄱㅣㅁ4ㅂㅏㅂ'
print(syllabify_many(jamos))

(':', '김', '4', '밥')


In [61]:
complex_jamos = '제2023年ㄷㅗ Dec 4ㅇㅣㄹ.'
print(syllabify_many(complex_jamos))

('제', '2', '0', '2', '3', '年', '도', ' ', 'D', 'e', 'c', ' ', '4', '일', '.')


In [62]:
print(syllabify_many('ㄱㅏㅁ23'))
print(syllabify_many('ㄷㅗㄹㄱㅣㅁ'))
print(syllabify_many('ㅂㅣㅂㅣㅁㅂㅏㅂ'))
print(syllabify_many('ㅋㅔㅇㅣㄹㄹㅣ-ㅎㅐㅁㅣㄹㅌㅓㄴ'))

('감', '2', '3')
('돌', '김')
('비', '빔', '밥')
('케', '일', '리', '-', '해', '밀', '턴')


### Q2. 임의의 문자열이 주어졌을 때 한글 음절 유형 판별하기 (최대 1점)



In [63]:
def get_syllable_types(string):
    chars = syllabify_many(string)
    
    # [hw14adv] Q2
    return tuple() # EDIT THIS LINE

#### Q2 힌트

1. `get_syllable_type()` 및 `ishangul()` 함수를 활용할 수 있다.

#### Q3 채점 기준

1. 'ㅇㅗㄱㅅㅜㅅㅜ!'처럼 음절로 표현 가능한 한글 자모와 특수문자로 이루어진 문자열에서 한글 음절 유형만을 뽑아낼 수 있다. (0.5점)
2. 'ㅂㅏㅂㅏㅁㅁㅁㅁㅁㅂㅏ'처럼 음절로 표현되지 못하는 한글을 포함한 문자열에서 자모 유형을 함께 뽑아낼 수 있다. (+0.5점)

In [64]:
get_syllable_types('얇은')

('GVCC', 'VC')

In [65]:
# 요구사항 1 (0.5점)
get_syllable_types('ㅇㅗㄱㅅㅜㅅㅜ!')

('VC', 'CV', 'CV')

In [66]:
get_syllable_types('ㅂㅏ밤ㅂㅏ')

('CV', 'CVC', 'CV')

In [67]:
# 요구사항 2 (+0.5점)
get_syllable_types('ㅂㅏ밤ㅁㅁㅁㅁㅁㅂㅏ')

('CV', 'CVC', 'C', 'C', 'C', 'C', 'C', 'CV')

### Q3. 예외 해결하기 (0.5점)

`get_syllable_types()` 함수가 잘 정의되었다면, 아래 예시와 같이 `get_syllable_type()`과 같은 유형이 나와야 한다.

In [68]:
bam = '밤'
print(get_syllable_type(bam))
print(get_syllable_types(bam))

CVC
('CVC',)


그런데 'ᄇᆡᆨ'과 같은 일부 옛한글의 경우 두 함수의 결과물이 다르게 나온다.

In [69]:
baek = 'ᄇᆡᆨ'
print(get_syllable_type(baek))
print(get_syllable_types(baek))

CVVC
('C', 'VV', 'C')


이 문제를 해결하기 위해 아래의 두 함수를 수정하라.

In [70]:
def to_jungseong(jamo):
    try:        
        jname = name(jamo)

        # [hw14Adv] Q3
        # DO SOMETHING HERE
        # DO SOMETHING HERE
        
        return lookup(jname.replace('LETTER', 'JUNGSEONG'))
    
    except:
        return jamo

def to_letter(jamo):
    try:
        jname = name(jamo)

        # [hw14Adv] Q3
        # DO SOMETHING HERE
        # DO SOMETHING HERE
        
        return lookup(re.sub('(?:CHO|JUNG|JONG)SEONG', 'LETTER', jname))
    
    except:
        return jamo

#### Q3 힌트

1. 아래의 차트에서 문자의 이름을 잘 살펴보라.

   + https://unicode.org/charts/PDF/U1100.pdf HANGUL JAMO
   + https://unicode.org/charts/PDF/U3130.pdf HANGUL COMPATIBILITY JAMO

#### Q3 요구사항

1. 아래의 예시에서 두 함수의 결과물이 같으면 된다. (0.5점)

In [71]:
jamos = 'ᄇᆡᆨ'
print(get_syllable_type(jamos))
print(get_syllable_types(jamos))

CVVC
('CVVC',)
